In [1]:
import pandas as pd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving SMSSpamCollection.csv to SMSSpamCollection.csv


In [8]:
import io
messages = pd.read_csv(io.BytesIO(uploaded['SMSSpamCollection.csv']), sep = '\t', names = ["label","message"])

In [9]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
# Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []


In [15]:
# now start by removing all unnecessary words i.e. punctuations using regular expression

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
# bag of words is made using countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()  # bag of words

In [21]:
# converting categorical into numerical i.e. (0, 1)
y = pd.get_dummies(messages['label'])  

# removing one column in the output class as only one can classify very well
y = y.iloc[:,1].values  

In [25]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [26]:
# training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [27]:
# testing model
y_pred = spam_detect_model.predict(X_test)

In [28]:
# evaluating model using confusion matrix
# confusion matrix gives the count of true/false positives/negatives in a 2x2 matrix

from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)

In [29]:
confusion_m

array([[946,   9],
       [  8, 152]])

In [30]:
# accuracy score

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9847533632286996

In [41]:
# msg = ["Hey you won a lottery of 1 million , hurry claim it using this promo code now or you will loose all money"]
msg_input = input()
import numpy as np
msg_input = [msg_input]
vect = cv.transform(msg_input).toarray()
res = spam_detect_model.predict(vect)
if res == 1:
    print("Spam Alert !")
else:
    print("Just a Ham")

hey buddy, let's celebrate this success of yours!1
Just a Ham
